# Step 1 - Climate Analysis and Exploration

In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Analyze datasets

In [ ]:
# Find the most recent date in the data set.
first_row = session.query(Measurement).first()
first_row.__dict__


In [ ]:
first_row = session.query(Station).first()
first_row.__dict__

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

# Exploratory Precipitation Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 

# Finding the most recent date 

maxdate = session.query(func.max(Measurement.date)).scalar()
print(maxdate)

In [ ]:
# Calculate the date one year from the last date in data set.
countofrows = session.query(Measurement).\
    filter(Measurement.date >= "2016-08-23").filter(Measurement.date <= "2017-08-23").count()
print(countofrows)

In [ ]:
# Filtered the results to get 1 year of data, from the most recent date
stmt = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    group_by(Measurement.date).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    order_by(Measurement.date.desc()).statement

In [ ]:
# Filtered the results to get 1 year of data, from the most recent date
stmt = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    order_by(Measurement.date.asc()).statement

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column

precipbydatedf = pd.read_sql_query(stmt, session.bind)
precipbydatedf = precipbydatedf.dropna()
precipbydatedf = precipbydatedf.set_index('date')
precipbydatedf = precipbydatedf.rename(columns={"sum_1": "prcp"})
precipbydatedf.head(10)

In [ ]:
# rename the column for the sum of the prcp 
precipbydatedf = precipbydatedf.rename(columns={"sum_1": "prcp"})

In [ ]:
# Sort the dataframe by date
precipbydatedf.sort_values(by=['date'], ascending = 'true')
precipbydatedf.head(10)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

line_chart = precipbydatedf.plot(title = "Precipitation by Date", rot = 90)
line_chart.set_xlabel("Date")
line_chart.set_ylabel("Inches")
plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipbydatedf.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset

countofstations = session.query(Station).count()
print(countofstations)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
stmt1 = session.query(Measurement.station, func.count(Measurement.prcp)).\
    group_by(Measurement.station).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    order_by(Measurement.station.desc()).statement


In [ ]:
stationdf = pd.read_sql_query(stmt1, session.bind)
stationdf = stationdf.dropna()
stationdf = stationdf.set_index('station')
stationdf = stationdf.rename(columns={"count_1": "reading count"})
stationdf.head(10)

In [ ]:
# Sort the dataframe by date
stationdf.sort_values(by=['reading count'],ascending=False)
stationdf.head(10)

In [ ]:
# Using the most active station id from the previous query, calculate the lowest temperature.

lowesttemp = session.query(func.min(Measurement.tobs)).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    filter(Measurement.station == "USC00519397").scalar()

print(lowesttemp)

In [ ]:
# Using the most active station id from the previous query, calculate the highest temperature.

highesttemp = session.query(func.max(Measurement.tobs)).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    filter(Measurement.station == "USC00519397").scalar()

print(highesttemp)

In [ ]:
# Using the most active station id from the previous query, calculate the average temperature.

avgtemp = session.query(func.avg(Measurement.tobs)).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    filter(Measurement.station == "USC00519397").scalar()

print(avgtemp)

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and

stmt2 = session.query(Measurement.tobs).\
    filter(Measurement.date >= "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").\
    filter(Measurement.station == "USC00519397").statement

stationUSC00519397df = pd.read_sql_query(stmt2, session.bind)
stationUSC00519397df = stationUSC00519397df.dropna()
stationUSC00519397df = stationUSC00519397df.rename(columns={"tobs": "Temperature"})
stationUSC00519397df.head(10)


In [ ]:
# plot the results as a histogram

hist = stationUSC00519397df.hist(bins=12)


# Close session

In [ ]:
# Close Session
session.close()